# Elements of Programming Interviews
## Binary Trees
### Track 6: 10.1, 10.2, 10.4, 10.8, 10.10, 10.11, 10.14

### 10.1 - Test If a Binary Tree Is Balanced
>A binary tree is said to be balanced if for each node in the tree, the difference in the height of it's left and right subtrees is at most one. A perfect binary tree is balanced, as is a complete binary tree.
<img src='Images/balanced_tree.png'>

In [1]:
class BT(object):
    def __init__(self, data, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right
balanced_tree = BT(314, BT(6, BT(271, BT(28), BT(0)), BT(561, right=BT(3, BT(17)))), BT(6, BT(2, right=BT(1, BT(401,right=BT(641)), BT(257))), BT(271, right=BT(28))))
unbalanced_tree = BT(1, BT(2, BT(3, BT(4))), BT(5))
#Unbalanced Tree
#        1
#       2 5
#      3
#     4

In [2]:
def post_order(node):
    if node is not None:
        post_order(node.left)
        post_order(node.right)
        print node.data,
    return
post_order(unbalanced_tree)

4 3 2 5 1


In [3]:
def height(node):
    if node is None:
        return -1
    return max(height(node.left), height(node.right)) + 1

In [4]:
def is_balanced(root):
    height_diff = abs(height(root.left) - height(root.right))
    return height_diff <= 1

In [5]:
is_balanced(unbalanced_tree), is_balanced(balanced_tree)

(False, True)

### 10.2 - Check If a Binary Tree Is Symetric
>A binary tree is symetric if you can draw a vertical line throught the root and then the left subtree is the mirror image of the right subtree. The node values must be equal as well.
>>                   *
              *      *
            *   *  *   *

In [6]:
def is_symetric(root):
    return root is None or check_symetry(root.left, root.right)
    
def check_symetry(lnode, rnode):
    if lnode is None and rnode is None:
        return True
    else:
        if lnode is not None and rnode is not None:
            return (lnode.data == rnode.data and
                    check_symetry(lnode.left, rnode.right) and
                    check_symetry(lnode.right, rnode.left))
    return False

In [7]:
symetric_tree = BT(1, BT(2, BT(3, BT(4), BT(5))), BT(2,left=None, right=BT(3, BT(5), BT(4))))
#           1
#          2 2
#        3     3
#      4  5  4  5
asymetric_tree = unbalanced_tree

(is_symetric(symetric_tree), is_symetric(None), is_symetric(asymetric_tree))

(True, True, False)

In [8]:
def recursion_test(n):
    if n == 1:
        return False
    elif n > 1:
        recursion_test(n-1)
        return True
recursion_test(15)

True

### 10.3 - Compute the Lowest Common Ancestor Between Two Nodes **NO PARENT ATTR**
>Given two nodes in a binary tree, design an algorith, that computes their LCA. Any two nodes in a binary tree have a cmmon ancestor, namely the root. the LCA of any two nodes in a binary tree is the node furthest from the root that is an ancestor of both nodes. 

<img src='Images/lca.png' height='450' width='450'>

First Approach: 
* Get the unique path from each node to the root in the form of a python set. 
    * Can be done with a BFS
* Then take the intersection of both sets, wherein the last element will be the LCA.

In [9]:
import os
os.chdir('/home/william/Python/Algorithms/Data Structures')
from stack import Stack

In [10]:
def get_path_from_root(root, search_node):
    if root is None or search_node is None:
        return None
    S = Stack()
    S.push(root)
    visited = []
    while not S.empty():
        curr = S.top()
        if curr == search_node:
            break
            
        if curr.left is not None and curr.left not in visited:
            S.push(curr.left)
            visited.append(curr.left)
        elif curr.right is not None and curr.right not in visited:
            S.push(curr.right)
            visited.append(curr.right)
        else:
            S.pop()
    path = []
    while not S.empty():
        path.append(S.top())
        S.pop()
    return path[::-1]

def get_lca(root, node1, node2):
    node1_path = set(get_path_from_root(root, node1))
    node2_path = set(get_path_from_root(root, node2))
    common_nodes = node1_path.intersection(node2_path)
    return list(common_nodes)[-1]

In [11]:
tree = symetric_tree
#           1
#          2 2
#        3     3
#      4  5  4  5
node1 = tree.left.left.right
node2 = tree.right
#common node is root -- 1
get_lca(tree, node1, node2).data

1

In [12]:
node1 = tree.right.right.left
node2 = tree.right.right.right
get_lca(tree, node1, node2).data

3

A *possible* optimization could be to do a traversal of the entire left subtree, and see if both nodes are in that tree. 
* If both nodes are in that left tree, then find the paths of both and then find their common last node -- the LCA.
* If only one node was found in the left subtree, then the LCA must be the root. Consequently, the paths of the nodes won't have to be calculated -- saving some time and space.

The books solution is below.

In [13]:
def get_lca(root, node1, node2):
    return LCA_helper(root, node1, node2)[0]
def LCA_helper(tree, node1, node2):
    if tree is None:
        return [None, 0]
    #check the left subtree
    left_result = LCA_helper(tree.left, node1, node2)
    if left_result[1] == 2:
        #both nodes found
        return left_result
    right_result = LCA_helper(tree.right, node1, node2)
    if right_result[1] == 2:
        return right_result
    num_target_nodes = left_result[1] + right_result[1] + (tree is node1 or tree is node2)
    res = [None, num_target_nodes]
    res[0] = tree if num_target_nodes == 2 else None  
    return res
    
        

In [14]:
get_lca(tree, node1, node2)

### 10.8 - Implement An Inorder Traversal With O(1) Space
>The direct implementation of an inorder traversal using recursion has O(h) space complexity, where $h$ is the height of the tree. Recursion can be removed with an explicit stack, but the space complexity remains O(h).
>
Write a nonrecursive program for computing the inorder traversal sequence for a binary tree. Assume nodes have parent fields.

<img src='Images/traversals.jpg'>

In [15]:
#extremely inefficient but basic method of creating a BT w/ parent attr in nodes
class Node():
    def __init__(self, data, parent=None, left=None, right=None):
        self.data = data
        self.parent = parent
        self.left = left
        self.right = right
nodes = {ch : Node(ch) for ch in 'ABCDEFGHI'}
nodes['H'].parent = nodes['E'];nodes['I'].parent = nodes['E'] 
nodes['E'].left = nodes['H']; nodes['E'].right = nodes['I']
nodes['D'].parent = nodes['B']; nodes['E'].parent = nodes['B']
nodes['B'].left = nodes['D']; nodes['B'].right = nodes['E']
nodes['B'].parent = nodes['A']; nodes['C'].parent = nodes['A']
nodes['A'].left = nodes['B']; nodes['A'].right = nodes['C']
nodes['F'].parent = nodes['C']; nodes['G'].parent = nodes['C']
nodes['C'].left = nodes['F']; nodes['C'].right = nodes['G']
root = nodes['A']

The logic for this solution is pretty straight forward.
* The starting point should be the leftmost desecndant of the root
* The next node has two possible cases
    * The current node has a right child, so go print the leftmost descendant of that subtree
    * The current node has no right child, so go print the rightmost ancestor that is a left child

In [16]:
def leftmost_descendant(node):
    while node.left is not None:
        node = node.left
    return node

def is_left_child(node):
    if node is None or node.parent is None:
        return False
    return node is node.parent.left

def rightmost_ancestor(node):
    #rightmost ancestor of closest node that is a leftchild
    while not is_left_child(node):
        if node.parent is None:
            return None
        node = node.parent
    
    return node.parent
        
def in_order(root):
    curr = leftmost_descendant(root)
    while curr is not None:
        print curr.data,
        if curr.right is not None:
            curr = leftmost_descendant(curr.right)
        else:
            if is_left_child(curr):
                curr = curr.parent
            else:
                curr = rightmost_ancestor(curr)
        

In [17]:
in_order(root)

D B H E I A F C G


### 10.10 - Reconstruct A Binary Tree From Traversal Data
>Many different binary trrees yield the same sequence of keys in an inorder, preorder, or postorder traversal. However, given an inorder traversal and one of any two other traversal orders of a binary tree, there exists a unique binary tree that yield those orders, assuming each node holds a distinct key. For example, the unique binary tree whose 
* preorder traversal sequence is ${H,B,F,E,A,C,D,G,I}$ is:
* inorder traversal sequence is ${F,B,A,E,H,C,D,I,G}$ 

<img src='Images/reconstruct.png'>

>Given an inorder traversal sequence and a preorder traversal sequence of a binary tree, write a program to reconstruct the tree. Assume each node has a unique key.

* The first item in the preoreder sequence is the root, so start with that.
    * In the inorder seq. data, $[0:IndexOfRoot]$ are the left subtree nodes
* The second node in the preorder seq. is $B$, notice that any element appearing before $B$ in the inorder seq. must be a left child  of $B$.

Iterate through the preorder, creating a subtree rooted at each element that is iterated over.
At each element, you're constructing a tree with a root, and have to find each child

PRE------${H,B,F,E,A,C,D,G,I}$----------------------------IN------${F,B,A,E,H,C,D,I,G}$ 
Split

In [70]:
inorder = [c for c in 'FBAEHCDIG']
preorder = [c for c in 'HBFEACDGI']

def reconstruct_tree(preorder, p_ix, inorder):
    root_val = preorder[p_ix] if p_ix < len(preorder) else None
    split_ix = inorder.index(root_val) if root_val in inorder else None
    if split_ix is not None:
        left_subtree = inorder[0:split_ix]
        right_subtree = inorder[split_ix + 1:]
    #now make the left child recursively
    tree = Node(root_val)
    #there is only a left child if the next value that appears in preorder is also in left_subtree
    if (p_ix + 1) < len(preorder) and preorder[p_ix+1] in left_subtree:
        tree.left = reconstruct_tree(preorder, p_ix+1, left_subtree)
    else:
        tree.left = None
    #we split at split_ix, thus anything to the right is a node of the right subtree
    rchild_start_ix = p_ix + split_ix + 1
    if rchild_start_ix < len(preorder) and preorder[rchild_start_ix] in right_subtree:
        tree.right = reconstruct_tree(preorder, rchild_start_ix, right_subtree)
    else: tree.right = None
        
    return tree

In [68]:
def pre_order(node):
    if node is not None:
        print node.data,
        post_order(node.left)
        post_order(node.right)
    return

In [69]:
t = reconstruct_tree(preorder, 0, inorder)
pre_order(t)

H B F E A C D G I
